<a href="https://colab.research.google.com/github/ShivaP69/CalibrationBasedOnEmotion/blob/main/emotion%26customizelambda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torch

In [ ]:
import pickle
import os
import time
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
ratings=pd.read_csv("/content/gdrive/MyDrive/ratings.csv",parse_dates=['timestamp'])

In [ ]:
tagsdf=pd.read_csv("/content/gdrive/MyDrive/tagsdf.csv") # tagsdf

In [ ]:
tags=pd.read_csv("/content/gdrive/MyDrive/tag.csv") #tags

In [ ]:
x= tags['movieId'].tolist()

In [ ]:
title_col = 'title'
genre_col = 'genres'

df_item = pd.read_csv('/content/gdrive/MyDrive/movie.csv')
df_item = df_item[df_item[genre_col] != '(no genres listed)']  # eliminate movies that had no genre information attached
print('dimension: ', df_item.shape)

dimension:  (27032, 3)


In [ ]:
user_col = 'userId'
item_col = 'movieId'
value_col = 'rating'
time_col = 'timestamp'

df= ratings[ratings['rating'] >= 4.0].copy()
ratings = df.merge(df_item, on='movieId')

In [ ]:
kaggle_df=ratings

In [ ]:
kaggle_df=kaggle_df.merge(tagsdf,on='movieId')

In [ ]:
kaggle_df

,Unnamed: 0,userId,movieId,rating,timestamp
0,236,3,1,4.0,1999-12-11 13:36:47
2,238,3,32,4.0,1999-12-11 13:14:07
3,239,3,50,5.0,1999-12-11 13:13:38
6,242,3,175,5.0,1999-12-11 13:32:13
8,244,3,223,5.0,1999-12-11 13:20:44
...,...,...,...,...,...
6027296,19999790,138491,1093,4.0,2009-03-04 01:36:27
6027305,19999799,138491,2857,5.0,2009-03-04 01:39:02
6027307,19999801,138491,3186,5.0,2009-03-04 01:38:28
6027309,19999803,138491,4128,4.0,2009-03-04 01:37:27


In [ ]:
from sklearn.model_selection import train_test_split
training_data, testing_data = train_test_split(kaggle_df, test_size=0.01, random_state=25)
training_data,validation_data=train_test_split(training_data, test_size=0.05,random_state=25)


In [ ]:
kaggle_df=pd.read_csv("/content/gdrive/MyDrive/ratings.csv",parse_dates=['timestamp'])
kaggle_df= kaggle_df[kaggle_df['rating'] >= 4.0].copy()

In [ ]:
kaggle_df = kaggle_df.merge(df_item, on='movieId')
df_id_title= kaggle_df[["movieId",'title']].drop_duplicates()

In [ ]:
from sklearn.model_selection import train_test_split
trainin_data=pd.read_csv('/content/gdrive/MyDrive/training_data.csv')
testing_data=pd.read_csv('/content/gdrive/MyDrive/testing_data.csv')
validation_data=pd.read_csv('/content/gdrive/MyDrive/validation_data.csv')

In [ ]:
#trainin_data,validation_data=train_test_split(trainin_data, test_size=0.05,random_state=25)

In [ ]:
#validation_data.to_csv('/content/gdrive/MyDrive/validation_data.csv')

In [ ]:
df_id_title= kaggle_df[["movieId",'title']].drop_duplicates()

In [ ]:
x=validation_data['userId'].tolist()
y=testing_data['userId'].tolist()
data=[]
for i in x:
  if i in y:
    data.append(i)

In [ ]:
validation_data= validation_data.loc[validation_data['userId'].isin(data)]

In [ ]:
validation_data

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,userId,movieId,rating,timestamp,title,genres
0,405943,1178299,9610158,66492,1249,1,2004-01-18 03:40:52,"Femme Nikita, La (Nikita) (1990)",Action|Crime|Romance|Thriller
1,1230513,1096279,2908096,19724,1597,1,2000-07-28 18:39:33,Conspiracy Theory (1997),Drama|Mystery|Romance|Thriller
2,1265470,733988,18483402,127954,2291,1,2008-03-06 14:26:13,Edward Scissorhands (1990),Drama|Fantasy|Romance
3,2097078,782616,13669516,94433,2959,1,2004-11-12 12:27:54,Fight Club (1999),Action|Crime|Drama|Thriller
4,522167,1787930,18359695,127063,5528,1,2011-04-17 20:45:50,One Hour Photo (2002),Drama|Thriller
...,...,...,...,...,...,...,...,...,...
120230,395910,780298,8916785,61633,2959,1,2015-02-01 20:42:12,Fight Club (1999),Action|Crime|Drama|Thriller
120231,692366,2873712,7845243,54107,6579,1,2014-11-15 01:42:07,"One, Two, Three (1961)",Comedy
120232,591084,2089938,19959365,138200,6218,1,2006-03-02 21:12:10,Bend It Like Beckham (2002),Comedy|Drama|Romance
120233,141022,3002786,19049965,131822,62000,1,2010-03-17 23:58:29,"Steamroller and the Violin, The (Katok i skrip...",Drama


In [ ]:
train_ratings= trainin_data
test_ratings=testing_data

In [ ]:
train_ratings.loc[:, 'rating'] = 1

train_ratings.sample(5)

,Unnamed: 0,Unnamed: 0.1,userId,movieId,rating,timestamp,title,genres
1987153,785242,19235998,133113,2959,1,2009-03-30 15:22:00,Fight Club (1999),Action|Crime|Drama|Thriller
213153,316857,14658884,101257,1261,1,1999-12-12 15:45:52,Evil Dead II (Dead by Dawn) (1987),Action|Comedy|Fantasy|Horror
3474,358767,13114616,90599,1544,1,1999-10-17 16:49:27,"Lost World: Jurassic Park, The (1997)",Action|Adventure|Sci-Fi|Thriller
840658,2391080,11130290,76911,55908,1,2008-12-28 18:51:21,"Man from Earth, The (2007)",Drama|Sci-Fi
1766854,1151070,16679454,115383,508,1,1996-08-13 05:41:51,Philadelphia (1993),Drama


In [ ]:
from torch.utils.data import Dataset,DataLoader
class MovieLensTrainDataset(Dataset):
    """MovieLens PyTorch Dataset for Training

    Args:
        ratings (pd.DataFrame): Dataframe containing the movie ratings
        all_movieIds (list): List containing all movieIds

    """

    def __init__(self, ratings, all_movieIds):
        self.users, self.items, self.labels = self.get_dataset(ratings, all_movieIds)

    def __len__(self):
        return len(self.users)

    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.labels[idx]

    def get_dataset(self, ratings, all_movieIds):
        users, items, labels = [], [], []

        user_item_dict=train_ratings.groupby('userId')['movieId'].apply(list).to_dict()


        num_negatives = 4
        for u in user_item_dict:
            users.append(u)
            for k in user_item_dict[u]:
              items.append(k)
              labels.append(1)
            for _ in range(num_negatives):
                negative_item = np.random.choice(all_movieIds)
                if negative_item not in user_item_dict[u]:
                  users.append(u)
                  items.append(negative_item)
                  labels.append(0)

        return torch.tensor(users), torch.tensor(items), torch.tensor(labels)

In [ ]:
pip install pytorch-lightning


In [ ]:
import pytorch_lightning as pl
import torch.nn as nn
class NCF(pl.LightningModule):
    """ Neural Collaborative Filtering (NCF)

        Args:
            num_users (int): Number of unique users
            num_items (int): Number of unique items
            ratings (pd.DataFrame): Dataframe containing the movie ratings for training
            all_movieIds (list): List containing all movieIds (train + test)
    """

    def __init__(self, num_users, num_items, ratings, all_movieIds):
        super().__init__()
        self.user_embedding = nn.Embedding(num_embeddings=num_users, embedding_dim=8)
        self.item_embedding = nn.Embedding(num_embeddings=num_items, embedding_dim=8)
        self.fc1 = nn.Linear(in_features=16, out_features=64)
        self.fc2 = nn.Linear(in_features=64, out_features=32)
        self.output = nn.Linear(in_features=32, out_features=1)
        self.ratings = ratings
        self.all_movieIds = all_movieIds

    def forward(self, user_input, item_input):
      # Pass through embedding layers
        user_embedded = self.user_embedding(user_input)
        item_embedded = self.item_embedding(item_input)

        # Concat the two embedding layers
        vector = torch.cat([user_embedded, item_embedded], dim=-1)

        # Pass through dense layer
        vector = nn.ReLU()(self.fc1(vector))
        vector = nn.ReLU()(self.fc2(vector))

        # Output layer
        pred = nn.Sigmoid()(self.output(vector))

        return pred
    def training_step(self, batch, batch_idx):
        user_input, item_input, labels = batch
        predicted_labels = self(user_input, item_input)
        loss = nn.BCELoss()(predicted_labels, labels.view(-1, 1).float())
        tensorboard_logs = {'train_loss': loss}
        return {'loss': loss, 'log': tensorboard_logs}


    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters())

    def train_dataloader(self):
        return DataLoader(MovieLensTrainDataset(self.ratings, self.all_movieIds),
                          batch_size=512, num_workers=4)


In [ ]:
num_users = kaggle_df['userId'].max()+1
num_items = kaggle_df['movieId'].max()+1

all_movieIds = kaggle_df['movieId'].unique()

model = NCF(num_users, num_items, train_ratings, all_movieIds)

In [ ]:
all_movieIds = kaggle_df['movieId'].unique()

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger
import torch
#from pytorch_lightning.loggers.neptune import NeptuneLogger



logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = pl.Trainer(max_epochs=10, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, checkpoint_callback=True,logger=logger)

trainer.fit(model)

trainer.save_checkpoint("deep_embd.ckpt")



"""trainer = pl.Trainer(max_epochs=5, gpus=1, reload_dataloaders_every_epoch=True,
                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)

trainer.fit(model)"""

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:50: LightningDeprecationWarning: `reload_dataloaders_every_epoch` is deprecated in v1.4 and will be removed in v1.6. Please use `reload_dataloaders_every_n_epochs` in Trainer.
  "`reload_dataloaders_every_epoch` is deprecated in v1.4 and will be removed in v1.6."
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name           | Type      | Params
---------------------------------------------
0 | user_embedding | Embedding | 1.1 M 
1 | item_embedding | Embedding | 1.0 M 
2 | fc1            | Linear    | 1.1 K 
3 | fc2            | Linear    | 2.1 K 
4 | output         | Linear    | 33    
---------------------------------------------
2.2 M     Trainable params
0         Non-trainable params
2.2 M     Total params
8.642     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.p

Training: -1it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:406: LightningDeprecationWarning: One of the returned values {'log'} has a `grad_fn`. We will detach it automatically but this behaviour will change in v1.6. Please detach it manually: `return {'loss': ..., 'something': something.detach()}`
  f"One of the returned values {set(extra.keys())} has a `grad_fn`. We will detach it automatically"


'trainer = pl.Trainer(max_epochs=5, gpus=1, reload_dataloaders_every_epoch=True,\n                     progress_bar_refresh_rate=50, logger=False, checkpoint_callback=False)\n\ntrainer.fit(model)'

In [ ]:
test_user_item_dict= test_ratings.groupby('userId')['movieId'].apply(list).to_dict()

In [ ]:
user_interacted_items = trainin_data.groupby('userId')['movieId'].apply(list).to_dict()

In [ ]:
validation_dict= validation_data.groupby('userId')['movieId'].apply(list).to_dict()

In [ ]:
#User-item pairs for testing
#test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))
test_user_item_dict= test_ratings.groupby('userId')['movieId'].apply(list).to_dict()

# Dict of all items that are interacted with by each user
user_interacted_items = trainin_data.groupby('userId')['movieId'].apply(list).to_dict() ########## training_data

candidate_items={}
interacted_items_kaggle={}
interacted_items=[]
test_items={}
hits = []
rows=[]
predicted_labels={}
top20_items={}
for u in test_user_item_dict.keys():
  if u in user_interacted_items.keys():

    not_interacted_items = set(all_movieIds) - set(user_interacted_items[u])
    selected_not_interacted = list(np.random.choice(list(not_interacted_items), 99))
    test_items[u] = selected_not_interacted + test_user_item_dict[u]
    #tt[u]=selected_not_interacted + test_user_item_dict[u]
    k=len(test_items[u])


    predicted_labels[u] = np.squeeze(model(torch.tensor([u]*k),torch.tensor(test_items[u])).detach().numpy()) # could be interpreted as score
    top20_items[u] = [test_items[u][i] for i in np.argsort(predicted_labels[u])[::-1][0:20].tolist()] # what items in test_items are in 20 most probable predicted labels
    candidate_items[u]=[test_items[u][i] for i in np.argsort(predicted_labels[u])[::-1][0:50].tolist()] # all items in test_items by order
    interacted_items_kaggle[u]=df_id_title[df_id_title['movieId'].isin(user_interacted_items[u])]['title'].tolist() # title
    #calib_recommend(candidate_items,,topn=20,lmbda=0.8)
    for i in test_user_item_dict[u]:
        if i in top20_items[u]:
          hits.append(1)
        else:
          hits.append(0)


a_file=open("/content/gdrive/MyDrive/top20_items_test.pkl","wb")
pickle.dump(top20_items,a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/candidate_items_test.pkl", "wb")
pickle.dump(candidate_items, a_file)
a_file.close()


a_file = open("/content/gdrive/MyDrive/predicted_labels_test.pkl", "wb")
pickle.dump(predicted_labels, a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/interacted_items_kaggle_test.pkl", "wb")
pickle.dump(interacted_items_kaggle, a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/test_items_test.pkl", "wb")
pickle.dump(test_items, a_file)
a_file.close()




print("The Hit Ratio @ 20 is {:.2f}".format(np.average(hits)))

The Hit Ratio @ 20 is 0.56


In [ ]:


a_file = open("/content/gdrive/MyDrive/top20_items_test.pkl", "rb")
top20_items = pickle.load(a_file)
a_file.close()
a_file = open("/content/gdrive/MyDrive/candidate_items_test.pkl", "rb")
candidate_items = pickle.load(a_file)
a_file.close()
a_file = open("/content/gdrive/MyDrive/predicted_labels_test.pkl", "rb")
predicted_labels = pickle.load(a_file)
a_file.close()
a_file = open("/content/gdrive/MyDrive/interacted_items_kaggle.pkl", "rb")
interacted_items_kaggle = pickle.load(a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/test_items_test.pkl", "rb")
test_items = pickle.load(a_file)
a_file.close()

In [ ]:
#User-item pairs for testing
#test_user_item_set = set(zip(test_ratings['userId'], test_ratings['movieId']))
validation_dict= validation_data.groupby('userId')['movieId'].apply(list).to_dict()

# Dict of all items that are interacted with by each user
user_interacted_items = trainin_data.groupby('userId')['movieId'].apply(list).to_dict() ########## training_data

candidate_items={}
interacted_items_kaggle={}
interacted_items=[]
test_items={}
hits = []
rows=[]
predicted_labels={}
top20_items={}
for u in validation_dict.keys():
    rows.append(u)
    if u in user_interacted_items.keys():
      #not_interacted_items = set(all_movieIds) - set(user_interacted_items[u])
      selected_not_interacted =list(np.random.choice(list(not_interacted_items), 200))
      #test_items[u]= validation_dict[u]

      test_items[u] = selected_not_interacted + validation_dict[u]
      #tt[u]=selected_not_interacted + test_user_item_dict[u]

      k=len(test_items[u])


      predicted_labels[u] = np.squeeze(model(torch.tensor([u]*k),torch.tensor(test_items[u])).detach().numpy()) # could be interpreted as score
      top20_items[u] = [test_items[u][i] for i in np.argsort(predicted_labels[u])[::-1][0:20].tolist()] # what items in test_items are in 20 most probable predicted labels
      candidate_items[u]=[test_items[u][i] for i in np.argsort(predicted_labels[u])[::-1][0:50].tolist()] # all items in test_items by order
      interacted_items_kaggle[u]=df_id_title[df_id_title['movieId'].isin(user_interacted_items[u])]['title'].tolist() # title
      #calib_recommend(candidate_items,,topn=20,lmbda=0.8)
      for i in validation_dict[u]:
          if i in top20_items[u]:
            hits.append(1)
          else:
            hits.append(0)







a_file=open("/content/gdrive/MyDrive/top20_items.pkl","wb")
pickle.dump(top20_items,a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/candidate_items.pkl", "wb")
pickle.dump(candidate_items, a_file)
a_file.close()


a_file = open("/content/gdrive/MyDrive/predicted_labels.pkl", "wb")
pickle.dump(predicted_labels, a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/interacted_items_kaggle.pkl", "wb")
pickle.dump(interacted_items_kaggle, a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/test_items.pkl", "wb")
pickle.dump(test_items, a_file)
a_file.close()

print(hits)
print("The Hit Ratio @ 20 is {:.2f}".format(np.average(hits)))

In [ ]:


a_file = open("/content/gdrive/MyDrive/top20_items.pkl", "rb")
top20_items = pickle.load(a_file)
a_file.close()
a_file = open("/content/gdrive/MyDrive/candidate_items.pkl", "rb")
candidate_items = pickle.load(a_file)
a_file.close()
a_file = open("/content/gdrive/MyDrive/predicted_labels.pkl", "rb")
predicted_labels = pickle.load(a_file)
a_file.close()
a_file = open("/content/gdrive/MyDrive/interacted_items_kaggle.pkl", "rb")
interacted_items_kaggle = pickle.load(a_file)
a_file.close()

a_file = open("/content/gdrive/MyDrive/test_items.pkl", "rb")
test_items = pickle.load(a_file)
a_file.close()

In [ ]:
#interacted # validationd data


def interaction_ditribution_val(tagsdf,user_interacted_items):

    pr=0.001
    beta=0.001


    finall_interatced_ditribution={}
    for user_id in user_interacted_items_val.keys() :
      distr={}
      interacted_distr={}

      df_user = tagsdf.loc[tagsdf['movieId'].isin(user_interacted_items[user_id])]
      df_user= df_user.drop(['Unnamed: 0', 'movieId'], axis=1)

      for col in df_user.columns:
        interacted_distr[col]=(df_user[col].sum())* (1-beta) + beta*pr



      interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}
      k= len(interacted_distr.keys())
      for emotion, score in interacted_distr.items():
        normalize_emotions = round(score /(k) , 3)
        distr[emotion] = normalize_emotions



      finall_interatced_ditribution[user_id]=distr
    return finall_interatced_ditribution


user_interacted_items_val = validation_data.groupby('userId')['movieId'].apply(list).to_dict() ########## training_data
interacted_distr_val= interaction_ditribution_val(tagsdf,user_interacted_items_val)

In [ ]:




def compute_kl_divergence(interacted_distr, reco_distr, alpha=0.01):
  kl_div = 0.
  for emotion, score in interacted_distr.items():
      reco_score = reco_distr.get(emotion, 0.)
      if score==0 and reco_score==0:
        kl_div+=0
      else:
        reco_score = (1 - alpha) * reco_score + alpha * score
        kl_div += score * np.log2(score / reco_score)

  return kl_div


def q_distr(list_items):

    interacted_distr={}

    df_user = tagsdf.loc[tagsdf['movieId'].isin(list_items)]
    df_user= df_user.drop(['Unnamed: 0', 'movieId'], axis=1)

    for col in df_user.columns:
      interacted_distr[col]=(df_user[col].sum())

    interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}

    return interacted_distr


kl_divergence={}
for user_id in top20_items.keys():

  reco_distribution = q_distr(top20_items[user_id])
  kl_divergence[user_id] = compute_kl_divergence(interacted_distr[user_id],reco_distribution)





In [ ]:
import heapq
from operator import itemgetter
topitems = heapq.nlargest(500, kl_divergence.items(), key=itemgetter(1))  # Use .iteritems() on Py2
topitemsasdict = dict(topitems)

In [ ]:
rows=topitemsasdict.keys()

In [ ]:
len(topitemsasdict.keys())

500

In [ ]:
rows= topitemsasdict.keys()
#rows=(random.sample(rows, 500))

In [ ]:
for i in rows:
  if i not in topitemsasdict.keys():
    print(False)

In [ ]:
import heapq
from operator import itemgetter
topitems = heapq.nsmallest(500, kl_divergence.items(), key=itemgetter(1))  # Use .iteritems() on Py2
topitemsasdict = dict(topitems)

In [ ]:
import random
rows= topitemsasdict.keys()
rows=(random.sample(rows, 500))

In [ ]:
import datetime
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

def difference(lst1,lst2):
  return list(set(lst1) - set(lst2))

def Recall(test_user_item_dict,topn,final_calibrated_list_kaggle):


  recall_second_reco=0
  i=1

  for user_id in final_calibrated_list_kaggle.keys() :

    Likes =test_user_item_dict[user_id]
  # calculate recall (reco & likes)


    TP_reco=intersection(Likes,final_calibrated_list_kaggle[user_id])
    FN_reco=difference(Likes,final_calibrated_list_kaggle[user_id])

    #recall_init_reco= len(TP_reco)/(len(TP_reco)+len(FN_reco)) if (len(TP_reco)+len(FN_reco))!=0 else 1
    recall_init_reco=len(TP_reco)/len(test_user_item_dict[user_id]) if len(test_user_item_dict[user_id])!=0 else 1
    #recall_init_reco=len(TP_reco)/20
    recall_second_reco+=recall_init_reco
    print(datetime.datetime.now())
    print("\nRecall of simple recommendation by model in" + str(i), " ", "is : ", recall_second_reco / i)
    i+=1
  recall_finall_reco = recall_second_reco/len(rows)

  #recall_sum[user_id]=recall_finall_reco

  print("recall of recommendation : ",recall_finall_reco )


topn=20


Recall(test_user_item_dict,topn,final_calibrated_list_kaggle)



"""lmbda=0.8
topn=20
model=bpr

rows=finall_test_rows

Recall(rows,inter,user_item_train,user_item_test,topn,bpr,index2item,item_mapping,reco_items)"""





In [ ]:
# find reco to nlargest when no calibration
simple_list={}
for key in rows:
  simple_list[key]=top20_items[key]

In [ ]:
# calculate the average of KL when lambda=0
finall=[]
for k,v in kl_divergence.items():
  if k in rows:
    finall.append(v)
sum(finall)/len(rows)

0.37995520079379236

In [ ]:
# /content/gdrive/MyDrive/final_calibrated_list_kaggle_0.5_largest.pkl
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_variance_largest.pkl", "rb")
calib = pickle.load(a_file)
a_file.close()


In [ ]:
user_interacted_items_val.keys()

In [ ]:
rows=[11, 17, 22, 23, 24, 29, 44, 49, 53, 55, 62, 63, 69,]

In [ ]:




def compute_kl_divergence(interacted_distr, reco_distr, alpha=0.01):
  kl_div = 0.
  for emotion, score in interacted_distr.items():
      reco_score = reco_distr.get(emotion, 0.)
      if score==0 and reco_score==0:
        kl_div+=0
      else:
        reco_score = (1 - alpha) * reco_score + alpha * score
        kl_div += score * np.log2(score / reco_score)

  return kl_div


def q_distr(list_items):

    interacted_distr={}

    df_user = tagsdf.loc[tagsdf['movieId'].isin(list_items)]
    df_user= df_user.drop(['Unnamed: 0', 'movieId'], axis=1)

    for col in df_user.columns:
      interacted_distr[col]=(df_user[col].sum())

    interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}

    return interacted_distr

calib=simple_list
kl={}
for user_id in calib.keys():

  reco_distribution = q_distr(calib[user_id])
  kl[user_id] = compute_kl_divergence(interacted_distr[user_id],reco_distribution)





In [ ]:
finall=[]
for k,v in kl.items():
  if k in rows:
    finall.append(v)
sum(finall)/len(rows)

0.380649976700031

In [ ]:
#interacted

#emotion
def interaction_ditribution(users,tagsdf,user_interacted_items):

    pr=0.001
    beta=0.001


    finall_interatced_ditribution={}
    for user_id in users:
      distr={}
      interacted_distr={}

      df_user = tagsdf.loc[tagsdf['movieId'].isin(user_interacted_items[user_id])]
      df_user= df_user.drop(['Unnamed: 0', 'movieId'], axis=1)

      for col in df_user.columns:
        interacted_distr[col]=(df_user[col].sum())* (1-beta) + beta*pr



      interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}
      k= len(interacted_distr.keys())
      for emotion, score in interacted_distr.items():
        normalize_emotions = round(score /(k) , 3)
        distr[emotion] = normalize_emotions



      finall_interatced_ditribution[user_id]=distr
    return finall_interatced_ditribution


In [ ]:
users=validation_data['userId'].tolist()

In [ ]:
users=validation_data['userId'].tolist()
interacted_distr=interaction_ditribution(users,tagsdf, user_interacted_items)
a_file = open("/content/gdrive/MyDrive/interacted_distr_val.pkl", "wb")
pickle.dump(interacted_distr, a_file)
a_file.close()

In [ ]:
df=pd.read_csv("/content/gdrive/MyDrive/df.csv")

In [ ]:
#interacted

# genres
def interaction_ditribution(users,tagsdf,user_interacted_items):

    pr=0.001
    beta=0.001


    finall_interatced_ditribution={}
    for user_id in users:
      distr={}
      interacted_distr={}

      df_user = tagsdf.loc[tagsdf['movieId'].isin(user_interacted_items[user_id])]
      df_user= df_user.drop(['Unnamed: 0', 'movieId','title'], axis=1)

      for col in df_user.columns:
        interacted_distr[col]=(df_user[col].sum())* (1-beta) + beta*pr


      # between zero and one

      interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}
      # sum to one
      k= len(interacted_distr.keys())
      for genre, score in interacted_distr.items():
        normalize_genre = round(score /(k) , 3)
        distr[genre] =  normalize_genre



      finall_interatced_ditribution[user_id]=distr
    return finall_interatced_ditribution

users=validation_data['userId'].tolist()
interacted_distr=interaction_ditribution(users,df, user_interacted_items)
a_file = open("/content/gdrive/MyDrive/interacted_distr_val_genre.pkl", "wb")
pickle.dump(interacted_distr, a_file)
a_file.close()

In [ ]:
interacted_distr=interaction_ditribution(test_user_item_dict,tagsdf, user_interacted_items)
a_file = open("/content/gdrive/MyDrive/interacted_distr.pkl", "wb")
pickle.dump(interacted_distr, a_file)
a_file.close()

In [ ]:
a_file = open("/content/gdrive/MyDrive/interacted_distr_val_genre.pkl", "rb")
interacted_distr = pickle.load(a_file)
a_file.close()

In [ ]:
P={"a":1, "b":2, "c":3}
mp=sum(P.values())/len(P)
d2 = {key: (abs(value-mp))**2 for key,value in P.items()}
print(sum(d2.values()))

2.0


In [ ]:
# define rows randomly
import random
rows= test_user_item_dict.keys()
rows=(random.sample(rows, 500))

In [ ]:
# cusomaized lambda


def variance(rows,P):
  lmbda={}
  for user_id in rows:
   mp= sum(P[user_id].values())/len(P[user_id])
   d2 = {key: abs((value-mp))**2 for key,value in P[user_id].items()}
   lmbda[user_id]= 1- (sum(d2.values())/len(P[user_id]))
  return lmbda



In [ ]:
with open("/content/gdrive/MyDrive/rows.txt", "w") as f:
    for s in rows:
        f.write(str(s) +"\n")

In [ ]:
score = []
with open("/content/gdrive/MyDrive/rows.txt", "r") as f:
  for line in f:
    score.append(int(line.strip()))

In [ ]:
rows=score

In [ ]:
class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
            reco_score = (1 - alpha) * reco_score + alpha * score
            kl_div += score * np.log2(score / reco_score)

      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)
      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,tagsdf,lmbda):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      calib_reco=[]
      calib_distr={}
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              df_calib =tagsdf.loc[tagsdf['movieId'].isin(probable_calib)]

              df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
              for col in df_calib.columns:
                calib_distr[col]=df_calib[col].sum()
              calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}
              k= len(calib_distr.keys())
              for emotion, score in calib_distr.items():
                      normalize_emotions = round(score /(k) , 3)
                      calib_distr[emotion] = normalize_emotions




              st = set(

              )
              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:
                #print('predi',predicted_labels[i])

                scores.append(predicted_labels[i])
              #print(scores)




              total_score= sum(scores)

              #print("total_score", total_score)

              utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
              if utility > max_utility:
                  max_utility = utility
                  best_item = item

          calib_reco.append(best_item)

      return calib_reco



  def calibrated_final_list(self,interacted_items_kaggle,candidate_items,test_items,predicted_labels,rows,interacted_distr,tagsdf,lmbda):

    calibrated_recommendations={}
    for user_id in rows:
      if user_id in interacted_distr.keys() & test_items.keys():
        calibrated_recommendations[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],tagsdf,lmbda[user_id])
    return calibrated_recommendations


CalibratedList = CalibratedListProduction(20)
#rows= conversion(finall_test_rows,finall_test)

lmbda= variance(rows, interacted_distr)

final_calibrated_list_kaggle =CalibratedList.calibrated_final_list(interacted_items_kaggle,candidate_items,test_items ,predicted_labels,rows,interacted_distr,tagsdf,lmbda)
print("completed")
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_variance.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()






/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:67: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: divide by zero encountered in log2
  app.launch_new_instance()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in double_scalars
  app.launch_new_instance()


completed


In [ ]:
tagsdf

In [ ]:
#emotion
Unnamed: 0
df1=tagsdf.drop('Unnamed: 0',axis=1)
df1.set_index("movieId", drop=True, inplace=True)
dictionary = df1.to_dict(orient="index")

In [ ]:
class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
            reco_score = (1 - alpha) * reco_score + alpha * score
            kl_div += score * np.log2(score / reco_score)

      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)
      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,tagsdf,lmbda):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibration_list=[]
      calib_distr={}
      calib_reco=[]
      v={}
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              calib_distr={k:dictionary[k] for k in probable_calib if k in dictionary}

              h=sum((Counter(d) for d in calib_distr.values()), Counter())
              v= {k: v / total for total in (sum(h.values()),) for k, v in h.items()}

              k= len(v.keys())
              for emotion, score in v.items():
                      normalize_emotions = round(score /(k) , 3)
                      v[emotion] = normalize_emotions



              st = set(probable_calib)
              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:
                #print('predi',predicted_labels[i])

                scores.append(predicted_labels[i])
              #print(scores)




              total_score= sum(scores)

              #print("total_score", total_score)

              utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
              if utility > max_utility:
                  max_utility = utility
                  best_item = item

          calib_reco.append(best_item)

      return calib_reco



  def calibrated_final_list(self,interacted_items_kaggle,candidate_items,test_items,predicted_labels,rows,interacted_distr,tagsdf,lmbda):

    calibrated_recommendations={}
    for user_id in rows:
      if user_id in interacted_distr.keys() & test_items.keys():
        calibrated_recommendations[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],tagsdf,lmbda)
    return calibrated_recommendations


CalibratedList = CalibratedListProduction(20)
#rows= conversion(finall_test_rows,finall_test)

lmbda= 0.8

final_calibrated_list_kaggle =CalibratedList.calibrated_final_list(interacted_items_kaggle,candidate_items,test_items ,predicted_labels,rows,interacted_distr,tagsdf,lmbda)
print("completed")
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()






completed


In [ ]:
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_variance_largest.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()


In [ ]:
#val distribution
def compute_kl_divergence_val(int, val_distr, alpha=0.01):
  kl_div = 0.
  for emotion, score in int.items():
      reco_score = val_distr.get(emotion, 0.)
      if score==0 and reco_score==0:
        kl_div+=0
      else:
        reco_score = (1 - alpha) * reco_score + alpha * score
        kl_div += score * np.log2(score / reco_score)

  return kl_div

In [ ]:
# genre
# validation data distribution and its difference to history of user
##### for genre
users=validation_data['userId'].tolist()


validation_dict= validation_data.groupby('userId')['movieId'].apply(list).to_dict()
KLD_val={}
val_distr={}
for user_id in users:
  if user_id in interacted_distr.keys():
    distr={}
    interacted_distr_val={}

    df_user = df.loc[tagsdf['movieId'].isin(validation_dict[user_id])]
    df_user= df_user.drop(['Unnamed: 0', 'movieId','title'], axis=1)

    for col in df_user.columns:
      interacted_distr_val[col]=df_user[col].sum()



    interacted_distr_val = {k: v / total for total in (sum(interacted_distr_val.values()),) for k, v in interacted_distr_val.items()}
    k= len(interacted_distr_val.keys())
    for genre, score in interacted_distr_val.items():
      normalize_genre = round(score /(k) , 3)
      distr[genre] = normalize_genre
    val_distr[user_id]=distr # distr of user in validation data
    KLD_val[user_id]= compute_kl_divergence_val(interacted_distr[user_id],val_distr[user_id]) # kld validation user and history




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log2
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars


In [ ]:

# validation data distribution and its difference to history of user

users=validation_data['userId'].tolist()


validation_dict= validation_data.groupby('userId')['movieId'].apply(list).to_dict()
KLD_val={}
val_distr={}
for user_id in users:
  if user_id in interacted_distr.keys():
    distr={}
    interacted_distr_val={}

    df_user = tagsdf.loc[tagsdf['movieId'].isin(validation_dict[user_id])]
    df_user= df_user.drop(['Unnamed: 0', 'movieId'], axis=1)

    for col in df_user.columns:
      interacted_distr_val[col]=df_user[col].sum()



    interacted_distr_val = {k: v / total for total in (sum(interacted_distr_val.values()),) for k, v in interacted_distr_val.items()}
    k= len(interacted_distr_val.keys())
    for emotion, score in interacted_distr_val.items():
      normalize_emotions = round(score /(k) , 3)
      distr[emotion] = normalize_emotions
    val_distr[user_id]=distr # distr of user in validation data
    KLD_val[user_id]= compute_kl_divergence_val(interacted_distr[user_id],val_distr[user_id]) # kld validation user and history




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: divide by zero encountered in log2
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
import random
rows=validation_dict.keys()
rows=(random.sample(rows, 500))

In [ ]:

#!!!!!!!!!!!!!!!!!
#genre # genre

#fisrt idea

class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
             if score!=0:
              reco_score = (1 - alpha) * reco_score + alpha * score
              kl_div += score * np.log2(score / reco_score)

      return kl_div
  def kl_divergence(self,p, q):
    return (p * np.log2(p/q))
    #for i in range(len(p)))


  def js_divergence(self,p, q):
    m = 0.5 * (p + q)
    return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)

  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility




  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,df,custom,val_distr,validation_dict,KLD_val):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibration_list=[]
      calib_distr={}
      max=-np.inf
      Best_lmbda=0
      for lmbda in [0,0.3,0.5,0.8,0.9]:
        calib_reco=[]
        for _ in range(self.topn):
            max_utility = -np.inf
            for item in candidate:  # candidate items


                total_score=0
                if item in calib_reco: # ignore duplicate items
                    continue

                probable_calib = calib_reco + [item]


                df_calib =df.loc[tagsdf['movieId'].isin(probable_calib)]

                df_calib= df_calib.drop(['Unnamed: 0', 'movieId','title'], axis=1)
                for col in df_calib.columns:
                  calib_distr[col]=df_calib[col].sum()
                calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}

                k= len(calib_distr.keys())
                for emotion, score in calib_distr.items():
                        normalize_emotions = round(score /(k) , 3)
                        calib_distr[emotion] = normalize_emotions

                st = set(probable_calib)
                indexes= ([i for i, e in enumerate(test_items) if e in st])

                #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

                scores=[]
                for i in indexes:

                  scores.append(predicted_labels[i])
                total_score= sum(scores)

                #print("total_score", total_score)

                utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
                if utility > max_utility:
                    max_utility = utility
                    best_item = item
            calib_reco.append(best_item)

        df_calib =df.loc[df['movieId'].isin(calib_reco)]

        df_calib= df_calib.drop(['Unnamed: 0', 'movieId','title'], axis=1)
        distr_eval={}
        for col in df_calib.columns:
          distr_eval[col]=df_calib[col].sum()
        distr_eval= {k: v / total for total in (sum(distr_eval.values()),) for k, v in distr_eval.items()}

        k= len(distr_eval.keys())
        for emotion, score in distr_eval.items():
                normalize_emotions = round(score /(k) , 3)
                distr_eval[emotion] = normalize_emotions
        #KLD= compute_kl_divergence(interacted_distr,distr_eval)

        sum_score=0

        list_hit=[]
        for i in calib_reco:
          if i in validation_dict:
            list_hit.append(i)
        st = set(list_hit)
        indexes= ([i for i, e in enumerate(test_items) if e in st])
        for j in indexes:
          sum_score+= predicted_labels[j]


        KLD= self.compute_kl_divergence(interacted_distr,distr_eval)
        new_KLD= self.kl_divergence(KLD_val,KLD)



        MMR= ((1-lmbda)*sum_score) + (lmbda*new_KLD )

        if MMR>max:
          Best_lmbda=lmbda
          max=MMR
          print("Best",Best_lmbda)
      return  Best_lmbda



  def calibrated_final_list(self,candidate_items,test_items,predicted_labels,rows,interacted_distr,df,val_distr,validation_dict,KLD_val):

    calibrated_recommendations={}
    custom= variance(rows,interacted_distr)
    landa_best={}
    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:
       print('user',user_id)


       landa_best[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],df,custom[user_id],val_distr[user_id],validation_dict[user_id],KLD_val[user_id])

    return landa_best


CalibratedList = CalibratedListProduction(15)
#rows= conversion(finall_test_rows,finall_test)


my_landa =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,df,val_distr,validation_dict,KLD_val)
print("completed")
"""a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_val_genre.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()

"""




In [ ]:

#!!!!!!!!!!!!!!!!!


#fisrt idea

class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
             if score!=0:
              reco_score = (1 - alpha) * reco_score + alpha * score
              kl_div += score * np.log2(score / reco_score)

      return kl_div
  def kl_divergence(self,p, q):
    return (p * np.log2(p/q))
    #for i in range(len(p)))


  def js_divergence(self,p, q):
    m = 0.5 * (p + q)
    return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)

  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility




  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,tagsdf,custom,val_distr,validation_dict,KLD_val):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibration_list=[]
      calib_distr={}
      max=-np.inf
      Best_lmbda=0
      for lmbda in [0,0.3,0.5,0.8,0.9]:
        calib_reco=[]
        for _ in range(self.topn):
            max_utility = -np.inf
            for item in candidate:  # candidate items


                total_score=0
                if item in calib_reco: # ignore duplicate items
                    continue

                probable_calib = calib_reco + [item]


                df_calib =tagsdf.loc[tagsdf['movieId'].isin(probable_calib)]

                df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
                for col in df_calib.columns:
                  calib_distr[col]=df_calib[col].sum()
                calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}

                k= len(calib_distr.keys())
                for emotion, score in calib_distr.items():
                        normalize_emotions = round(score /(k) , 3)
                        calib_distr[emotion] = normalize_emotions

                st = set(probable_calib)
                indexes= ([i for i, e in enumerate(test_items) if e in st])

                #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

                scores=[]
                for i in indexes:

                  scores.append(predicted_labels[i])
                total_score= sum(scores)

                #print("total_score", total_score)

                utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
                if utility > max_utility:
                    max_utility = utility
                    best_item = item
            calib_reco.append(best_item)

        df_calib =tagsdf.loc[tagsdf['movieId'].isin(calib_reco)]

        df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
        distr_eval={}
        for col in df_calib.columns:
          distr_eval[col]=df_calib[col].sum()
        distr_eval= {k: v / total for total in (sum(distr_eval.values()),) for k, v in distr_eval.items()}

        k= len(distr_eval.keys())
        for emotion, score in distr_eval.items():
                normalize_emotions = round(score /(k) , 3)
                distr_eval[emotion] = normalize_emotions
        #KLD= compute_kl_divergence(interacted_distr,distr_eval)

        sum_score=0

        list_hit=[]
        for i in calib_reco:
          if i in validation_dict:
            list_hit.append(i)
        st = set(list_hit)
        indexes= ([i for i, e in enumerate(test_items) if e in st])
        for j in indexes:
          sum_score+= predicted_labels[j]


        KLD= self.compute_kl_divergence(interacted_distr,distr_eval)
        new_KLD= self.kl_divergence(KLD_val,KLD)



        MMR= ((1-lmbda)*sum_score) + (lmbda*new_KLD )

        if MMR>max:
          Best_lmbda=lmbda
          max=MMR
          print("Best",Best_lmbda)
      return  Best_lmbda



  def calibrated_final_list(self,candidate_items,test_items,predicted_labels,rows,interacted_distr,tagsdf,val_distr,validation_dict,KLD_val):

    calibrated_recommendations={}
    custom= variance(rows,interacted_distr)
    landa_best={}
    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:
       print('user',user_id)


       landa_best[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],tagsdf,custom[user_id],val_distr[user_id],validation_dict[user_id],KLD_val[user_id])

    return landa_best


CalibratedList = CalibratedListProduction(15)
#rows= conversion(finall_test_rows,finall_test)


my_landa =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,tagsdf,val_distr,validation_dict,KLD_val)
print("completed")
"""a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_0.8_smallest.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()

"""




In [ ]:
a_file = open("/content/gdrive/MyDrive/landa.pkl", "wb")
pickle.dump(my_landa, a_file)
a_file.close()

In [ ]:

#!!!!!!!!!!!!!!!!!

class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
             if score!=0:
              reco_score = (1 - alpha) * reco_score + alpha * score
              kl_div += score * np.log2(score / reco_score)

      return kl_div
  def kl_divergence(p, q):
    return sum(p[i] * log2(p[i]/q[i]) for i in range(len(p)))


  def js_divergence(self,p, q):
    m = 0.5 * (p + q)
    return 0.5 * kl_divergence(p, m) + 0.5 * kl_divergence(q, m)

  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility




  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,tagsdf,custom,val_distr,validation_dict):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibration_list=[]
      calib_distr={}
      max=-np.inf
      Best_lmbda=0
      for lmbda in [0,0.3,0.5,0.8,0.9]:
        calib_reco=[]
        for _ in range(self.topn):
            max_utility = -np.inf
            for item in candidate:  # candidate items


                total_score=0
                if item in calib_reco: # ignore duplicate items
                    continue

                probable_calib = calib_reco + [item]


                df_calib =tagsdf.loc[tagsdf['movieId'].isin(probable_calib)]

                df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
                for col in df_calib.columns:
                  calib_distr[col]=df_calib[col].sum()
                calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}

                k= len(calib_distr.keys())
                for emotion, score in calib_distr.items():
                        normalize_emotions = round(score /(k) , 3)
                        calib_distr[emotion] = normalize_emotions

                st = set(probable_calib)
                indexes= ([i for i, e in enumerate(test_items) if e in st])

                #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

                scores=[]
                for i in indexes:

                  scores.append(predicted_labels[i])
                total_score= sum(scores)

                #print("total_score", total_score)

                utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
                if utility > max_utility:
                    max_utility = utility
                    best_item = item
            calib_reco.append(best_item)

        df_calib =tagsdf.loc[tagsdf['movieId'].isin(calib_reco)]

        df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
        distr_eval={}
        for col in df_calib.columns:
          distr_eval[col]=df_calib[col].sum()
        distr_eval= {k: v / total for total in (sum(distr_eval.values()),) for k, v in distr_eval.items()}

        k= len(distr_eval.keys())
        for emotion, score in distr_eval.items():
                normalize_emotions = round(score /(k) , 3)
                distr_eval[emotion] = normalize_emotions
        #KLD= compute_kl_divergence(interacted_distr,distr_eval)

        sum_score=0
        print(calib_reco, "for this", lmbda)
        print(validation_dict)
        list_hit=[]
        for i in calib_reco:
          if i in validation_dict:
            list_hit.append(i)
        st = set(list_hit)
        indexes= ([i for i, e in enumerate(test_items) if e in st])
        for j in indexes:
          sum_score+= predicted_labels[j]

        print("sum", sum_score)

        KLD= self.compute_kl_divergence(val_distr,distr_eval)
        print("KLD",KLD)

        MMR= ((1-lmbda)*sum_score) + (lmbda*KLD )

        if MMR>max:
          Best_lmbda=lmbda
          max=MMR
          print("Best",Best_lmbda)
      return  Best_lmbda



  def calibrated_final_list(self,candidate_items,test_items,predicted_labels,rows,interacted_distr,tagsdf,val_distr,validation_dict):

    calibrated_recommendations={}
    custom= variance(rows,interacted_distr)
    landa_best={}
    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:
       print('user',user_id)


       landa_best[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],tagsdf,custom[user_id],val_distr[user_id],validation_dict[user_id])

    return landa_best


CalibratedList = CalibratedListProduction(15)
#rows= conversion(finall_test_rows,finall_test)


my_landa =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,tagsdf,val_distr,validation_dict)
print("completed")
"""a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_0.8_smallest.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()

"""




In [ ]:
def compute_kl_divergence(interacted_distr, reco_distr, alpha=0.01):
    kl_div = 0.
    for emotion, score in interacted_distr.items():
        reco_score = reco_distr.get(emotion, 0.)

        if score==0 and reco_score==0:
          kl_div+=0
        else:
          reco_score = (1 - alpha) * reco_score + alpha * score
          if score!=0:
            kl_div += score * np.log2(score / reco_score)

    return kl_div
val_distr ={'Happy': 0.012, 'Angry': 0.0, 'Surprise': 0.054, 'Sad': 0.0, 'Fear': 0.134}
distr_eval= {'Happy': 0.04, 'Angry': 0.009, 'Surprise': 0.031, 'Sad': 0.037, 'Fear': 0.083}
compute_kl_divergence(val_distr,distr_eval)

0.1133559001274367

In [ ]:
##### Test Our Idea
#with test data
class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
            reco_score = (1 - alpha) * reco_score + alpha * score
            kl_div += score * np.log2(score / reco_score)

      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility


  def compute_utility_new(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility,kl_div


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,tagsdf,lmbda):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibration_list=[]
      calib_distr={}
      calib_reco=[]
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              df_calib =tagsdf.loc[tagsdf['movieId'].isin(probable_calib)]

              df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
              for col in df_calib.columns:
                calib_distr[col]=df_calib[col].sum()
              calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}

              k= len(calib_distr.keys())
              for emotion, score in calib_distr.items():
                      normalize_emotions = round(score /(k) , 3)
                      calib_distr[emotion] = normalize_emotions

              st = set(probable_calib)
              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:

                scores.append(predicted_labels[i])
              total_score= sum(scores)

              #print("total_score", total_score)

              utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
              if utility > max_utility:
                  max_utility = utility
                  best_item = item
          calib_reco.append(best_item)





      return calib_reco



  def calibrated_final_list(self,candidate_items,test_items,predicted_labels,rows,interacted_distr,tagsdf,my_landa):

    calibrated_recommendations={}


    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:

      if user_id in interacted_distr.keys() & test_items.keys():
       calibrated_recommendations[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],tagsdf,my_landa[user_id])

    return calibrated_recommendations


CalibratedList = CalibratedListProduction(20)
#rows= conversion(finall_test_rows,finall_test)




final_calibrated_list_kaggle =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,tagsdf,my_landa)
print("completed")
"""a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_0.8_smallest.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()

"""




In [ ]:
df=pd.read_csv("/content/gdrive/MyDrive/df.csv")

In [ ]:
df1=df.drop(['Unnamed: 0','title'],axis=1)

In [ ]:
df1.set_index("movieId", drop=True, inplace=True)
dictionary = df1.to_dict(orient="index")

In [ ]:
predict_list=[54286,55069]

In [ ]:
new_dictionary={}
for key,value in dictionary.items():
  if key in predict_list:
    new_dictionary[key]=value

In [ ]:
from collections import Counter
h=sum((Counter(d) for d in new_dictionary.values()), Counter())


In [ ]:
calib_distr= {k: v / total for total in (sum(h.values()),) for k, v in h.items()}

In [ ]:

new_dictionary=sum((Counter(d) for d in new_dictionary.values()), Counter())
new_dictionary= {k: v / total for total in (sum(new_dictionary.values()),) for k, v in new_dictionary.items()}

In [ ]:
new_dictionary

{'Action': 0.16666666666666666,
 'Crime': 0.16666666666666666,
 'Drama': 0.5,
 'Thriller': 0.16666666666666666}

In [ ]:
z={k:dictionary[k] for k in predict_list if k in dictionary}

In [ ]:
a_file = open("/content/gdrive/MyDrive/landa_genre.pkl", "rb")
my_landa= pickle.load(a_file)
a_file.close()

In [ ]:
rows=my_landa.keys()

In [ ]:
##### Test Our Idea
#with test data.   #ganres # genres
from collections import Counter
class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn


  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:

            reco_score = (1 - alpha) * reco_score + alpha * score
            if score!=0:
              kl_div += score * np.log2(score / reco_score)


      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)


      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility


  def compute_utility_new(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility,kl_div


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,dictionary,lmbda):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibration_list=[]
      calib_distr={}
      calib_reco=[]
      v={}
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              calib_distr={k:dictionary[k] for k in probable_calib if k in dictionary}

              h=sum((Counter(d) for d in calib_distr.values()), Counter())
              v= {k: v / total for total in (sum(h.values()),) for k, v in h.items()}

              k= len(v.keys())
              for emotion, score in v.items():
                      normalize_emotions = round(score /(k) , 3)
                      v[emotion] = normalize_emotions

              st = set(probable_calib)

              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:

                scores.append(predicted_labels[i])
              total_score= sum(scores)


              #print("total_score", total_score)

              utility = self.compute_utility(v,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco

              if utility > max_utility:

                  max_utility = utility
                  best_item = item

          calib_reco.append(best_item)





      return calib_reco



  def calibrated_final_list(self,candidate_items,test_items,predicted_labels,rows,interacted_distr,dictionary,my_landa):

    calibrated_recommendations={}


    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:

      if user_id in interacted_distr.keys() & test_items.keys():
       calibrated_recommendations[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],dictionary,my_landa[user_id])

    return calibrated_recommendations


CalibratedList = CalibratedListProduction(20)





final_calibrated_list_kaggle =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,dictionary,my_landa)
print("completed")
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_genre.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()






completed


In [ ]:
##### Test Our Idea
#with test data.   #ganres # genres  # variance
from collections import Counter
class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn


  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for emotion, score in interacted_distr.items():
          reco_score = reco_distr.get(emotion, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:

            reco_score = (1 - alpha) * reco_score + alpha * score
            if score!=0:
              kl_div += score * np.log2(score / reco_score)


      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)


      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility


  def compute_utility_new(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """

      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)

      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility,kl_div


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,dictionary,lmbda):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      final_calibraation_list=[]
      calib_distr={}
      calib_reco=[]
      v={}
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              calib_distr={k:dictionary[k] for k in probable_calib if k in dictionary}

              h=sum((Counter(d) for d in calib_distr.values()), Counter())
              v= {k: v / total for total in (sum(h.values()),) for k, v in h.items()}

              k= len(v.keys())
              for emotion, score in v.items():
                      normalize_emotions = round(score /(k) , 3)
                      v[emotion] = normalize_emotions

              st = set(probable_calib)

              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:

                scores.append(predicted_labels[i])
              total_score= sum(scores)


              #print("total_score", total_score)

              utility = self.compute_utility(v,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco

              if utility > max_utility:

                  max_utility = utility
                  best_item = item

          calib_reco.append(best_item)





      return calib_reco



  def calibrated_final_list(self,candidate_items,test_items,predicted_labels,rows,interacted_distr,dictionary,custom):

    calibrated_recommendations={}


    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:

      if user_id in interacted_distr.keys() & test_items.keys():
       calibrated_recommendations[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],dictionary,custom[user_id])

    return calibrated_recommendations


CalibratedList = CalibratedListProduction(20)



custom=variance(rows,interacted_distr)

final_calibrated_list_kaggle =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,dictionary,custom)
print("completed")
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_genre_variance.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()






completed


In [ ]:


a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_variance.pkl", "rb")
final_calibrated_list_kaggle = pickle.load(a_file)
a_file.close()

In [ ]:
pip install ml_metrics

  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-py3-none-any.whl size=7845 sha256=4aa40a21649149a3d433ea94538cf2bdc20e19d0c0ed866b270656fb48026d10
  Stored in directory: /root/.cache/pip/wheels/56/41/5b/0c6d42b3604a5c823d8922564c4708f84962fa7f2f4facfa6d
Successfully built ml-metrics


In [ ]:

def get_ap_from_list(relevance_array):
    relevance_list_size = len(relevance_array)
    if relevance_list_size == 0:
        return 0.0

    relevant = 0
    for i in range(relevance_list_size):
        if relevance_array[i]:
            relevant += 1
        return relevant/relevance_list_size

def average_precision(test_items, reco_items):
    precision=0
    # reco_items_df[item_label] = reco_items_df[item_label].astype(str)
    for user in reco_items.keys():
      vector = [True if x in test_items[user] else False for x in reco_items[user]]

      precision+=get_ap_from_list(vector)
    return precision/len(reco_items.keys())



In [ ]:
average_precision(test_user_item_dict,final_calibrated_list_kaggle)

0.03775303643724723

In [ ]:
def precision(actual, predicted, k):
    act_set = set(actual)
    pred_set = set(predicted)

    result = len(act_set & pred_set) / 20
    return result

def MAP(actual,predicted,k):
  total=0

  for user in predicted.keys():
    #if user in actual.keys():

      p= precision(actual[user],predicted[user],k)
      total+=p
  return total/len(predicted.keys())


In [ ]:
print(MAP(test_user_item_dict,final_calibrated_list_kaggle,20))

0.47682186234817814


In [ ]:


# i think is not correct but is based on a git code
def get_rr_from_list(relevance_array):
    relevance_list_size = len(relevance_array)
    if relevance_list_size == 0:
        return 0.0
    for i in range(relevance_list_size):
        if relevance_array[i]:
            return 1 / (i + 1)
    return 0.0


def mrr(test_items, reco_items):
  precision=0
  for user in reco_items.keys():
      vector = [True if x in test_items[user] else False for x in reco_items[user]]

      precision+=get_rr_from_list(vector)
  return precision/(len(reco_items.keys()))

In [ ]:
mrr(test_user_item_dict,final_calibrated_list_kaggle)

0.7095930163055323

In [ ]:
def MRR(reco_items,test_user):
  rr_users=0
  for user in reco_items.keys():
    #if user in test_user.keys():
      rr=0
      for x in test_user[user]:
        if x in reco_items[user]:
          rr+=1/(reco_items[user].index(x) +1)
      rr_users+=rr/len(reco_items[user])
  return rr_users/len(reco_items.keys())



MRR(final_calibrated_list_kaggle,test_user_item_dict)


0.11459666604322756

In [ ]:

# show that the result is false for mrr of paper
def get_rr_from_list(relevance_array):
    relevance_list_size = len(relevance_array)
    if relevance_list_size == 0:
        return 0.0
    for i in range(relevance_list_size):
        if relevance_array[i]:
            return 1 / (i + 1)
    return 0.0

zz=[1,2,3,4,5]
ww=[2,4,5,1,8]
precision = [True if x in ww else False for x in zz]
print(precision)
print( get_rr_from_list(precision))



[True, True, False, True, True]
1.0


In [ ]:
rows= test_user_item_dict.keys() # all test users

In [ ]:
p={'Happy': 0.12192581056193642, 'Angry': 0.03334724539595396, 'Surprise': 0.2555231288475711, 'Sad': 0.25010420486094626, 'Fear': 0.33909961033359226}
q= {'Happy': 0.3846153846153846, 'Angry': 0.0, 'Surprise': 0.03678929765886287, 'Sad': 0.060200668896321065, 'Fear': 0.5183946488294314}


In [ ]:
r=ratings['movieId'].tolist()
df= tagsdf['movieId'].tolist()


In [ ]:
# MACE


# distribution of q

def q_distr(list_items):



      interacted_distr={}

      df_user = tagsdf.loc[tagsdf['movieId'].isin(list_items)]
      df_user= df_user.drop(['Unnamed: 0', 'movieId'], axis=1)

      for col in df_user.columns:
        interacted_distr[col]=(df_user[col].sum())

      interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}

      return interacted_distr


def CE(P,part_distr):


  d3 = {key: abs(P[key] - part_distr.get(key, 0)) for key in P}
  if d3=='nan':
    return 0

  else:
    return sum(d3.values())/len(P)


def ACE(P,q,N):
  all=0
  for i in range(N):
    part_distr= q_distr(q[0:i+1])

    all+= CE(P,part_distr)

  return all/N

def MACE (P,q,N):
  ace=0
  for user_id in q.keys():
    if user_id in interacted_distr.keys():

      ace+=ACE(P[user_id],q[user_id],N) # q here is a list of movies

  return ace/len(q.keys())


MACE(interacted_distr,final_calibrated_list_kaggle,20)





In [ ]:
# MACE

#genre
# distribution of q

def q_distr(list_items):



      interacted_distr={}

      df_user = df.loc[df['movieId'].isin(list_items)]
      df_user= df_user.drop(['Unnamed: 0', 'movieId','title'], axis=1)

      for col in df_user.columns:
        interacted_distr[col]=(df_user[col].sum())

      interacted_distr = {k: v / total for total in (sum(interacted_distr.values()),) for k, v in interacted_distr.items()}

      return interacted_distr


def CE(P,part_distr):


  d3 = {key: abs(P[key] - part_distr.get(key, 0)) for key in P}
  if d3=='nan':
    return 0

  else:
    return sum(d3.values())/len(P)


def ACE(P,q,N):
  all=0
  for i in range(N):
    part_distr= q_distr(q[0:i+1])

    all+= CE(P,part_distr)

  return all/N

def MACE (P,q,N):
  ace=0
  for user_id in q.keys():
    if user_id in interacted_distr.keys():

      ace+=ACE(P[user_id],q[user_id],N) # q here is a list of movies

  return ace/len(q.keys())


MACE(interacted_distr,final_calibrated_list_kaggle,20)





0.03999190283400769

In [ ]:
df=pd.read_csv('/content/gdrive/MyDrive/df.csv')

In [ ]:

#genre # different landa (0.8)
class CalibratedListProduction():
  def __init__(self,lmbda,topn):
    self.lmbda=lmbda
    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for genre, score in interacted_distr.items():
          reco_score = reco_distr.get(genre, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
            reco_score = (1 - alpha) * reco_score + alpha * score
            kl_div += score * np.log2(score / reco_score)

      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score):


      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)
      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - self.lmbda) * total_score - self.lmbda * kl_div
      return utility


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,dictionary):

      final_calibration_list=[]
      calib_distr={}
      calib_reco=[]
      v={}
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              calib_distr={k:dictionary[k] for k in probable_calib if k in dictionary}

              h=sum((Counter(d) for d in calib_distr.values()), Counter())
              v= {k: v / total for total in (sum(h.values()),) for k, v in h.items()}

              k= len(v.keys())
              for emotion, score in v.items():
                      normalize_emotions = round(score /(k) , 3)
                      v[emotion] = normalize_emotions

              st = set(probable_calib)

              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:

                scores.append(predicted_labels[i])
              total_score= sum(scores)


              #print("total_score", total_score)

              utility = self.compute_utility(v,interacted_distr,total_score) # add item to best_item if increase utility of calib_reco

              if utility > max_utility:

                  max_utility = utility
                  best_item = item

          calib_reco.append(best_item)





      return calib_reco

  def calibrated_final_list(self,candidate_items,test_items ,predicted_labels,rows,interacted_distr,dictionary):

    calibrated_recommendations={}


    #landa_list=[0,0.3,0.5,0.8,0.9]
    for user_id in rows:

      if user_id in interacted_distr.keys() & test_items.keys():
       calibrated_recommendations[user_id]=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],dictionary)

    return calibrated_recommendations


CalibratedList = CalibratedListProduction(0.8, 20)


final_calibrated_list_kaggle =CalibratedList.calibrated_final_list(candidate_items,test_items ,predicted_labels,rows,interacted_distr,dictionary)
print("completed")
a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()






/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in double_scalars


completed


In [ ]:
len(final_calibrated_list_kaggle)

500

In [ ]:
normal={}
for user_id in predicted_labels.keys():

  predicted_labels[user_id][:] = [x / sum(predicted_labels[user_id]) for x in predicted_labels[user_id]]



In [ ]:
sum(list(interacted_distr[4428].values()))

0.2

In [ ]:
# emotion


class CalibratedListProduction():
  def __init__(self,topn):

    self.topn=topn



  def compute_kl_divergence(self,interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.

      for emotion, score in interacted_distr.items():

          reco_score = reco_distr[emotion]
          #print(reco_score)
          if score==0 and reco_score==0:
            kl_div+=0
          else:

            reco_score = (1 - alpha) * reco_score + alpha * score
            kl_div += score * np.log2(score / reco_score)

      return kl_div



  def compute_utility(self,reco_distr, interacted_distr,total_score,lmbda):
      """
      Our objective function for computing the utility score for
      the list of recommended items.

      lmbda : float, 0.0 ~ 1.0, default 0.5
          Lambda term controls the score and calibration tradeoff,
          the higher the lambda the higher the resulting recommendation
          will be calibrated. Lambda is keyword in Python, so it's
          lmbda instead ^^
      """
      #print("reco", reco_distr)
      kl_div = self.compute_kl_divergence(interacted_distr,reco_distr)
      # kl divergence is the lower the better, while score is
      # the higher the better so remember to negate it in the calculation
      utility = (1 - lmbda) * total_score - lmbda * kl_div
      return utility


  def calib_recommend(self,candidate,interacted_distr,test_items,predicted_labels,tagsdf,lmbda):

      """
      start with an empty recommendation list,
      loop over the topn cardinality, during each iteration
      update the list with the item that maximizes the utility function.
      """
      calib_reco=[]
      calib_distr={}
      for _ in range(self.topn):
          max_utility = -np.inf
          for item in candidate:  # candidate items


              total_score=0
              if item in calib_reco: # ignore duplicate items
                  continue

              probable_calib = calib_reco + [item]


              df_calib =tagsdf.loc[tagsdf['movieId'].isin(probable_calib)]

              df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
              for col in df_calib.columns:
                calib_distr[col]=df_calib[col].sum()
              calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}
              k= len(calib_distr.keys())
              for emotion, score in calib_distr.items():
                normalize_emotions = round(score /(k) , 3)
                calib_distr[emotion] = normalize_emotions
              st = set(probable_calib)
              indexes= ([i for i, e in enumerate(test_items) if e in st])

              #indexes=[i for i, val in enumerate(test_items) if val in probable_calib]

              scores=[]
              for i in indexes:
                #print('predi',predicted_labels[i])

                scores.append(predicted_labels[i])
              #print(scores)

              total_score= sum(scores)

              #print("total_score", total_score)

              utility = self.compute_utility(calib_distr,interacted_distr,total_score,lmbda) # add item to best_item if increase utility of calib_reco
              if utility > max_utility:
                  max_utility = utility
                  best_item = item

          calib_reco.append(best_item)

      return calib_reco



  def calibrated_final_list(self,interacted_items_kaggle,candidate_items,test_items,predicted_labels,rows,interacted_distr,tagsdf):
    custom= variance(rows,interacted_distr)
    calibrated_recommendations={}
    max=-1000
    for user_id in rows:
      landa_value={}
      lmbda_list=[0.3,0.5,0,0.8,0.9,custom[user_id]]
      predicted=predicted_labels[user_id]
      for landa in lmbda_list:
        calib=self.calib_recommend(candidate_items[user_id], interacted_distr[user_id],test_items[user_id],predicted_labels[user_id],tagsdf,landa)
        scores=[]
        distr_here={}


        df_calib =tagsdf.loc[tagsdf['movieId'].isin(calib)]
        #print(df_calib)
        df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
        for col in df_calib.columns:
          distr_here[col]=df_calib[col].sum()

        distr_here= {k: v / total for total in (sum(distr_here.values()),) for k, v in distr_here.items()}

        for emotion, score in distr_here.items():
          normalize_emotions = round(score /(k) , 3)
          distr_here[emotion] = normalize_emotions

        kld= self.compute_kl_divergence(interacted_distr[user_id],distr_here)
        st = set(calib)
        #print(calib)
        indexes= ([i for i, e in enumerate(test_items[user_id]) if e in st])

        #print(indexes)
        for item in indexes:
          #print(item)
          scores.append(predicted[item])
        print(sum(scores))
        print(kld)
        comp= landa*(kld) + (1-landa) *sum(scores)
        if comp>max:
          max=comp
          calib_reco= calib
          max_landa=landa
          calibrated_recommendations[user_id]=calib_reco
          #print(calibrated_recommendations)
          landa_value[user_id]=landa
          print(landa_value)


    return calibrated_recommendations,land_value


CalibratedList = CalibratedListProduction(20)
#rows= conversion(finall_test_rows,finall_test)




final_calibrated_list_kaggle, values =CalibratedList.calibrated_final_list(interacted_items_kaggle,candidate_items,test_items ,predicted_labels,rows,interacted_distr,tagsdf)
print("completed")
"""a_file = open("/content/gdrive/MyDrive/final_calibrated_list_kaggle_0.8_smallest.pkl", "wb")
pickle.dump(final_calibrated_list_kaggle, a_file)
a_file.close()

"""




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: divide by zero encountered in log2
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars


0.2778987353667617
-0.46206802412243014
{22979: 0.3}
0.2778987353667617
-0.46206802412243014
0.2778987353667617
-0.46206802412243014
{22979: 0}


KeyboardInterrupt: ignored

In [ ]:
custom= variance(rows,interacted_distr)

In [ ]:
custom[91089]

0.9851612473192618

In [ ]:
def compute_kl_divergence(interacted_distr, reco_distr, alpha=0.01):
      kl_div = 0.
      for genre, score in interacted_distr.items():
          reco_score = reco_distr.get(genre, 0.)
          if score==0 and reco_score==0:
            kl_div+=0
          else:
            reco_score = (1 - alpha) * reco_score + alpha * score
            kl_div += score * np.log2(score / reco_score)

      return kl_div
predicted=predicted_labels[user]
score=[]
comp=[]
df_calib =tagsdf.loc[tagsdf['movieId'].isin(recommended)]
df_calib= df_calib.drop(['Unnamed: 0', 'movieId'], axis=1)
for col in df_calib.columns:
  calib_distr[col]=df_calib[col].sum()
  calib_distr= {k: v / total for total in (sum(calib_distr.values()),) for k, v in calib_distr.items()}
kld= compute_kl_divergence(interacted_distr,calib_distr)

for item in recommended:
  score.append(predicted[i])
comp[lmbda]= lmbda(kld) + (1-lmbda)sum(score)

